In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/tiva/PycharmProjects/HeadlineGenerator'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_dir: Path
    checkpoint: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from headlineGenerator.constants import *
from headlineGenerator.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILEPATH, params_filepath=PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArgs

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_dir=config.data_dir,
            checkpoint=config.checkpoint,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
import torch
from datasets import load_dataset, load_from_disk
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

[2024-03-17 06:20:19,360 || PyTorch version 2.2.0+cpu available.]


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.checkpoint)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.checkpoint).to(device)
        data_collator = DataCollatorForSeq2Seq(tokenizer, model)

        # loading data
        news_dataset = load_from_disk(self.config.data_dir)

        args = TrainingArguments(
            output_dir=self.config.root_dir,
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy,
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model=model,
            args=args,
            tokenizer=tokenizer,
            data_collator=data_collator,
            train_dataset=news_dataset["test"],
            eval_dataset=news_dataset["validation"]
        )

        trainer.train()

        # save model
        model.save_pretrained(os.path.join(self.config.root_dir, "t5-news-dataset-model"))
        # save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "t5-news-dataset-tokenizer"))
        

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-03-16 07:52:31,474 || yaml file : config.yaml loaded successfully]
[2024-03-16 07:52:31,564 || yaml file : params.yaml loaded successfully]
[2024-03-16 07:52:31,574 || created directory at artifacts]
[2024-03-16 07:52:31,596 || created directory at artifacts/model_trainer]


/home/tiva/PycharmProjects/HeadlineGenerator/venv/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss,Validation Loss


RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 132945920 vs 132945812